## 🎨🧩 Step 1: Public Artworks Data Modeling

### Import Libraries

In [1]:
# Import Libraries
import osmnx as ox # to fetch data from OpenStreetMap
import geopandas as gpd # to work with geospatial data
import pandas as pd
import numpy as np
import re
from geopy.geocoders import Nominatim
from tqdm import tqdm

In [10]:
full_df = pd.read_csv('/Users/biancaniemann/Documents/Webeet/Python/layered-populate-data-pool-da/museums_galleries/sources/csv_files/gallery_listings.csv')

In [12]:
part_df = full_df.head(20)

### Used reverse Geocoding and Nominatim to get Postal code, Street & house number info
    - Created postal_code, street and house_number columns to try get data from the latitude and longitude

In [14]:
tqdm.pandas()

# Initialize geocoder
geolocator = Nominatim(user_agent="gallery_locator")

# Define function to extract postal code and street
def get_postcode_and_street(row):
    try:
        location = geolocator.reverse((row['latitude'], row['longitude']), exactly_one=True)
        address = location.raw.get('address', {})
        postcode = address.get('postcode')
        street = address.get('road') or address.get('pedestrian') or address.get('footway') or address.get('street')
        # Get the first key-value pair in the address dict
        name = next(iter(address.values())) if address else None
        return pd.Series([postcode, street, name])
    except:
        return pd.Series([None, None, None])

# Apply to your DataFrame
part_df[['postal_code', 'street', 'name']] = part_df.progress_apply(get_postcode_and_street, axis=1)


100%|██████████| 20/20 [00:49<00:00,  2.46s/it]
/var/folders/yz/0rh2jmfd3xq7_g25xn3y55dh0000gn/T/ipykernel_69993/1571188592.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  part_df[['postal_code', 'street', 'name']] = part_df.progress_apply(get_postcode_and_street, axis=1)
/var/folders/yz/0rh2jmfd3xq7_g25xn3y55dh0000gn/T/ipykernel_69993/1571188592.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  part_df[['postal_code', 'street', 'name']] = part_df.progress_apply(get_postcode_and_street, axis=1)


In [16]:
part_df.head(20)

,gallery_id,gallery_name,house_number,street,neighbourhood_id,district_id,postal_code,website,opening_hours,wheelchair,fee,latitude,longitude,name
0,301107444,atelier achim kühn,None,Richterstraße,908,11009009,12524,NaN,NaN,NaN,NaN,52.409750,13.571342,Atelier Achim Kühn
1,370766098,galerie zandi,None,None,401,11004004,None,NaN,mo-fr 11:00-18:00; sa 11:00-16:00,no,False,52.503135,13.313631,None
2,410135935,studio für gestaltung,None,Bürknerstraße,801,11008008,12047,NaN,"th,fr 13:00-18:00; sa 12:00-16:00",yes,False,52.493327,13.427490,Studio für Gestaltung
3,410692505,la girafe,None,Glogauer Straße,202,11002002,10999,NaN,NaN,no,False,52.492914,13.437057,La Girafe
4,410745800,kronenboden,None,Schwedenstraße,106,11001001,13357,NaN,NaN,yes,NaN,52.555091,13.375326,kronenboden
5,511117580,galería lunasol,None,Schierker Straße,801,11008008,12051,NaN,"tu-th 11:00-22:00; fr,sa 11:00-23:00",yes,False,52.470012,13.432264,Galería Lunasol
6,543623547,errant sound,None,None,101,11001001,None,NaN,NaN,NaN,NaN,52.512362,13.419821,None
7,545738013,saarländische galerie,None,Am Festungsgraben,101,11001001,10117,https://www.saarlaendische-galerie.eu/,NaN,NaN,NaN,52.518549,13.395823,Saarländische Galerie
8,569103989,atelier freier fotografen,None,Weisbachstraße,201,11002002,10249,https://www.aff-galerie.de/,"sa,su 15:00-18:00",no,False,52.524655,13.451143,Atelier Freier Fotografen
9,573816270,NaN,None,Konrad-Wolf-Straße,1110,11011011,13055,NaN,NaN,NaN,NaN,52.541710,13.488287,99
